In [3]:
import wfdb
import numpy as np
from matplotlib import pyplot as plt
from helper_code import *
from utilities import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import ecg_plot


data_folder = "data/micro_code_sami/"
print('Finding the Challenge data...')
records = find_records(data_folder)
num_records = len(records)

if num_records == 0:
    raise FileNotFoundError('No data were provided.')

utilObject = UtilityFunctions("cpu")
sampling_rate = 400
leads_idxs = {'I': 0, 'II': 1, 'III':2, 'aVR': 3, 'aVL':4, 'aVF':5, 'V1':6, 'V2':7, 'V3':8, 'V4':9, 'V5':10, 'V6':11}

positive_counter = 0
negative_counter = 0
sami_trop_counter = 0
limit = 2
# Iterate over the records.
for i in range(num_records):
    recording_file = os.path.join(data_folder, records[i])
    header_file = os.path.join(data_folder, get_header_file(records[i]))
    header = load_header(recording_file)
    try:
        current_label= get_label(header)
    except Exception as e:
        print("Failed to load label, assigning 0")
        current_label = 0
    
    try:
        signal, fields = load_signals(recording_file)
    except Exception as e:
        print(f"Skipping {header_file} and associated recording  because of {e}")
        continue

    recording_full = utilObject.load_and_equalize_recording(signal, fields, header_file, sampling_rate)

    splitted = header.split("\n")
    source_info = [x for x in splitted if "Source" in x][0]

    if current_label == 0 and negative_counter <= limit and "CODE" in source_info:
        ecg_plot.plot_12(recording_full, sample_rate = 400, title = f"ECG 12 - HEALTHY-{source_info}")
        ecg_plot.save_as_png(f"ECG 12 - HEALTHY -{source_info}-{i}",'plots/')
        negative_counter += 1
        
    if current_label == 1 and positive_counter <= limit and "CODE" in source_info:
        ecg_plot.plot_12(recording_full, sample_rate = 400, title = f"ECG 12 - CHAGAS-{source_info}")
        ecg_plot.save_as_png(f"ECG 12 - CHAGAS-{source_info}-{i}",'plots/')
        positive_counter += 1
        
    if current_label == 1 and sami_trop_counter <= limit and "SaMi" in source_info:
        ecg_plot.plot_12(recording_full, sample_rate = 400, title = f"ECG 12 - CHAGAS-{source_info}")
        ecg_plot.save_as_png(f"ECG 12 - CHAGAS-{source_info}-{i}",'plots/')
        sami_trop_counter += 1
        
    if positive_counter > limit and negative_counter > limit and sami_trop_counter > limit:
        break
    

Finding the Challenge data...
